# 实验要求
1. 对数据集进行处理，转换成适合进行关联规则挖掘的形式；
2. 找出频繁模式；
3. 导出关联规则，计算其支持度和置信度;
4. 对规则进行评价，可使用Lift、卡方和其它教材中提及的指标, 至少2种；
5. 对挖掘结果进行分析；
6. 可视化展示。

获取数据集

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        #print(filename)
        if filename=="winemag-data-130k-v2.csv":
            data=pd.read_csv(os.path.join(dirname, filename))'''
data=pd.read_csv(r"/Users/lianqi/Desktop/winemag-data-130k-v2.csv")
print(type(data))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<class 'pandas.core.frame.DataFrame'>


查看数据集大致信息

In [2]:
print(data.info())
print("-"*80)
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


选择合适的属性列进行频繁模式和关联规则挖掘，这里我们选择country,points,price,province,region_1,region_2,variety,winery，将这些列提取出来形成新的数据集new_data。

In [3]:
new_data=data[["country","points","price","province","region_1","region_2","variety","winery"]]
print(new_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   country   129908 non-null  object 
 1   points    129971 non-null  int64  
 2   price     120975 non-null  float64
 3   province  129908 non-null  object 
 4   region_1  108724 non-null  object 
 5   region_2  50511 non-null   object 
 6   variety   129970 non-null  object 
 7   winery    129971 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 7.9+ MB
None


In [4]:
print(new_data.isnull().sum())
print(new_data.info())
new_data1=new_data.dropna(how="any")
new_data1.dropna(axis=0,inplace=False)
print(new_data1.isnull().sum())
print(new_data1.info())

country        63
points          0
price        8996
province       63
region_1    21247
region_2    79460
variety         1
winery          0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   country   129908 non-null  object 
 1   points    129971 non-null  int64  
 2   price     120975 non-null  float64
 3   province  129908 non-null  object 
 4   region_1  108724 non-null  object 
 5   region_2  50511 non-null   object 
 6   variety   129970 non-null  object 
 7   winery    129971 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 7.9+ MB
None
country     0
points      0
price       0
province    0
region_1    0
region_2    0
variety     0
winery      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50292 entries, 2 to 129967
Data columns (total 8 columns):
 #   Column    Non-Null Count  D

In [7]:
from progressbar import *
def new_set(dataset):
    ns=[]
    progress=ProgressBar()
    for data in progress(dataset):
        for item in data:
            #if [item] not in C1:
            ns.append([item])
    print("success")
    return [frozenset(item) for item in ns]

In [8]:
def filter_d(dataset, Ck):
    Ck_count = dict()
    for data in dataset:
        for cand in Ck:
            if cand.issubset(data):
                if cand not in Ck_count:
                    Ck_count[cand] = 1
                else:
                    Ck_count[cand] += 1

    num_items = float(len(dataset))
    return_list = []
    support_data = dict()
    for key in Ck_count:
        support  = Ck_count[key] / num_items
        if support >= 0.1:
              return_list.insert(0, key)
        support_data[key] = support
    return return_list, support_data

In [9]:
def concat(Lk, k):
        return_list = []
        len_Lk = len(Lk)
        for i in range(len_Lk):
            for j in range(i+1, len_Lk):
                L1 = list(Lk[i])[:k-2]
                L2 = list(Lk[j])[:k-2]
                L1.sort()
                L2.sort()
                if L1 == L2:
                    return_list.append(Lk[i] | Lk[j])
        return return_list

In [10]:
def apriori(dataset):
    C1 = new_set(dataset)
    dataset = [set(data) for data in dataset]
    L1, support_data = filter_d(dataset, C1)
    L = [L1]
    k = 2
    while len(L[k-2]) > 0:
        Ck = concat(L[k-2], k)
        Lk, support_k = filter_d(dataset, Ck)
        support_data.update(support_k)
        L.append(Lk)
        k += 1
    return L, support_data

In [11]:

freq_set, sup_rata = apriori(new_data1)

sup_rata_out = sorted(sup_rata.items(), key=lambda d: d[1], reverse=True)

100% (50292 of 50292) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


success
